# Семантическая Сегментация. Часть 2.

Мощная модель нейронной сети ASPP с делатационными свёртками которая увеличивает рецептивное поле для сегментации.

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf

## Модель ASPP

Создадим модель наследуясб от класса `tf.keras.Model`.

In [0]:
class ASPPBlock(tf.keras.Model):
    # ASPP блок создаём как отдельную модель (можно как слой)
    def __init__(self):
        super().__init__()
        # поканальная смесь всех двумерных признаков
        self.conv1 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')
        # группа из трёх делатационных свёрток с параметром (`dilation_rate`) - шаг между пикселями,
        # который мы семплируем на входном тензоре
        self.conv2 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=6, padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=12, padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=18, padding='same', activation='relu')
        # свёртка 1х1 из многих каналов делает на выходе 256 каналов
        self.conv5 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')

    def call(self, inp, is_training=False):
        # параллельно делаем 4 свёртки (в каждую подаём входной тензор)
        out1 = self.conv1(inp) 
        out2 = self.conv2(inp)
        out3 = self.conv3(inp)
        out4 = self.conv4(inp)
        # конкатенируем и получаем толстый тензор
        out = tf.concat([out1, out2, out3, out4], axis=3)
        # понижаем количество каналов толстого тензора с 4*256 до 256
        out = self.conv5(out)
        return out
    
class ASPPNet(tf.keras.Model):
    # реализация самой модели с мощным энкодером и слабым декодером, прдполагая, что 
    # блок ASPP сделал часть сложной работы, а декодеру останется повысить разрешение
    def __init__(self):
        super().__init__()
        # множество свёрток для извлечения признаков в энкодере
        self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv6 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv7 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv8 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv9 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv10 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')

        # свёртки для декодера
        self.conv11 = tf.keras.layers.Conv2D(48, (1, 1), padding='same', activation='relu')
        self.conv12 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv13 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        # из свёртки 1х1 получим выходную карту признаков которая отвечает за нашу сегментацию
        # активация None потому что сигмоид применяем отдельно
        self.conv14 = tf.keras.layers.Conv2D(1, (1, 1), padding='same', activation=None)

        self.maxpool = tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same')

        # мы не может создавать ASPP блок во время call, потому что у него есть внутри свёрточные
        # слои, а значит веса, которые нужно инициализировать, чтобы было сосотояние у сети
        self.aspp = ASPPBlock()

    # инференс, прямое распространение
    def call(self, x):
        # слои енкодера
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.maxpool(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.maxpool(out)
        out = self.conv5(out)
        out = self.conv6(out)
        # сохраняем промежуточный тензор, чтобы через skip connection передать в конец
        out_enc_mid = out
        out = self.maxpool(out)
        out = self.conv7(out)
        out = self.conv8(out)
        out = self.maxpool(out)
        out = self.conv9(out)
        out = self.conv10(out)

        # применяем ASPP
        out = self.aspp(out)

        # повышаем разрешение карты признаков с помощью билинейной интерполяции. Говорим, что хотим 
        # увеличить до размера tf.shape(out_enc_mid)[1:3] (1:3 - два пространственных измерения)
        out = tf.image.resize(out, tf.shape(out_enc_mid)[1:3], tf.image.ResizeMethod.BILINEAR)

        # приводим к фиксированному кол-ву каналов 48 свёрткой 1х1, перемешивая каналы каждый с каждым
        out_enc_mid = self.conv11(out_enc_mid)

        # конкатенируем с увеличиной картинкой
        out = tf.concat([out, out_enc_mid], axis=3)

        out = self.conv12(out)
        out = self.conv13(out)
        # свёртка 1х1 c одним выходным каналом. Свёртка перемешивает все выходные карты признаков и выдаёт
        # один выходной канал, которые по сути является картой сегментации
        out = self.conv14(out)

        # делаем ресайз до исходного размера x (входная картинка)
        out = tf.image.resize(out, tf.shape(x)[1:3], tf.image.ResizeMethod.BILINEAR)
        # применяем сигмоид
        out = tf.nn.sigmoid(out)
        return out
    
model = ASPPNet()